In [ ]:
import pandas as pd
from fpdf import FPDF
import glob, os
import json

In [ ]:
input_path1 = "../plots/"
input_path2 = "../text/"

In [ ]:
df = pd.read_csv('../input/test.csv')
df['FIPS'] = df['FIPS'].astype(str)
df['FIPS'] = df['FIPS'].apply(lambda s: '0' + s if len(s) == 4 else s)
df['County'] = df['County'].str.strip()

In [ ]:
df['FIPS'].nunique()

In [ ]:
type(df['FIPS'].iloc[0])

In [ ]:
# Custom class to overwrite the header and footer methods
class PDF(FPDF):
    def __init__(self):
        super().__init__()
    def header(self):
        self.set_font('Arial', '', 12)
        self.cell(0, 10, 'Header', 1, 1, 'C')
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', '', 12)
        self.cell(0, 10, 'Footer', 1, 0, 'C')


In [ ]:
def investment_summary(pdf, text, bullet_indent=15):
    pdf.set_font("Arial", size=8)

    for point in text.splitlines():
        if point.startswith("-"):
            pdf.set_x(bullet_indent)
        pdf.multi_cell(0, 5, point)

In [ ]:
def add_analytics(pdf, image_paths):
  for image in image_paths:
    pdf.image(image, w=180, h=100)  # Adjust width (w) and height (h) as needed
    pdf.add_page()  # Start a new page for each graph

In [ ]:
def add_analytics1(pdf, image_paths):
    size_of_image = 150
    # pdf.set_margin(5.0)
    for image in image_paths:
        pdf.image(image, w=size_of_image, x=((pdf.w-size_of_image)/2))
        pdf.ln(h=2.5)


In [ ]:
def create_report(county_and_state, llm_text, analytics_location_path=None):
    
    pdf = FPDF()
    pdf.add_page()
    
    pdf.set_text_color(r=50,g=108,b=175)
    pdf.set_font('Arial', 'B', 18)
    
    
    pdf.cell(w=0, h=10, txt="Verus-AI: " + county_and_state, ln=1,align='C')
    
    
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(w=0, h=10, txt="Investment Summary", ln=1,align='L')
    
    investment_summary(pdf, llm_text)

    pdf.set_text_color(r=50,g=108,b=175)
    pdf.set_font('Arial', 'B', 16)

    pdf.add_page()
    pdf.cell(w=0, h=10, txt="Analytics", ln=1,align='L')
    add_analytics1(pdf, analytics_location_path)
    
    
    
    
    
    
    pdf.output(f'./example1.pdf', 'F')

In [ ]:
df['County_Modified'] = df['County'].str.rstrip('County')
df['County_Modified'] = df['County_Modified'].str.replace('Census Area', '')
df['County_Modified'] = df['County_Modified'].str.strip()
dict = pd.Series(df.County_Modified.values,index=df.FIPS).to_dict()

In [ ]:
file_paths1 = []
for root, dirs, files in os.walk(input_path1):
    for name in files:
        if '.png' in name:
            file_paths1.append(os.path.join(root, name))

In [ ]:
file_paths2 = []
for root, dirs, files in os.walk(input_path2):
    for name in files:
        file_paths2.append(os.path.join(root, name))

In [ ]:
count = 0
for path in file_paths2:
    fips = path.split('/')[2]
    df_state = df[df['FIPS']==fips]
    state = df_state['State'].iloc[0]
    county_and_state = dict[fips] + ", " + state
    with open(path) as f:
        d = json.load(f)
        index = d.find(':')
        text = d[index+3:]
        find_graph = [x for x in file_paths1 if fips in x]
        create_report(county_and_state, text, find_graph)
        break
    